In [1]:
import rasterio
import numpy as np


tif_path = "MODCF_interannualSD.tif"


bounding_boxes = [
    ("Amazon Basin ", (-14.0, 5.0, -79.5, -60.5)),
    ("Central Africa", (-8.35,9.65, 6.95, 24.95)),
    ("Southeast Asia", (-75.25, -70.25,-57.25, -52.25)),
    ("Northen Australia", (-14.0, -11.5, 134.5, 137.0)),
    ("Eastern United States", (25.0, 50.0, -125.0, -70.0))
]

with rasterio.open(tif_path) as src:
    data = src.read(1)
    transform = src.transform
    nodata = src.nodata if src.nodata is not None else -9999
    height, width = data.shape

    for name, (lat_min, lat_max, lon_min, lon_max) in bounding_boxes:
        
        col1, row1 = ~transform * (lon_min, lat_max)
        col2, row2 = ~transform * (lon_max, lat_min)

        row_min = int(np.floor(min(row1, row2)))
        row_max = int(np.ceil(max(row1, row2)))
        col_min = int(np.floor(min(col1, col2)))
        col_max = int(np.ceil(max(col1, col2)))

        row_min = max(row_min, 0); row_max = min(row_max, height)
        col_min = max(col_min, 0); col_max = min(col_max, width)

        if row_min >= row_max or col_min >= col_max:
            print(f"{name}: Sd = N/A (Illegal index)")
            continue

        region_data = data[row_min:row_max, col_min:col_max]
        region_data = region_data[(region_data != nodata) & (~np.isnan(region_data))]

        if region_data.size < 10:
            print(f"{name}: Sd = N/A (Too few effective pixels)")
        else:
            mean_val = round(float(np.mean(region_data) / 100.0), 2)
            print(f"{name}: Sd = {mean_val}%")


Amazon Basin : Sd = 8.71%
Central Africa: Sd = 9.2%
Southeast Asia: Sd = 17.44%
Northen Australia: Sd = 13.46%
Eastern United States: Sd = 10.96%
